<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Modelo_Mix_Produ%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problema de mix de produção

Neste notebook, descrevemos como resolver um problema de decisão de mix de produção. O problema é formulado como um modelo de programação linear, e em seguida é resolvido com uso da biblioteca Pyomo para Python e o solver GLPK.

In [1]:
!pip install pyomo


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 KB 5.3 MB/s eta 0:00:00


In [2]:
import pyomo.environ as pyo
import numpy as np

##Dados do problema
c = np.array([2,3])  ##Lucro
A = np.array([[1.5, 3.0],
              [3.5, 4.0],
              [10.0, 7.0]])

b = np.array([180,280,700])    ##Horas disponíveis

In [ ]:
modelo = pyo.ConcreteModel()
##Variáveis de decisão
n = 2    ##Número de variáveis

modelo.x = pyo.Var([i for i in range(n)], domain=pyo.NonNegativeReals)

##Função objetivo
expr = sum(c[i]*modelo.x[i] for i in range(n))
modelo.obj = pyo.Objective(expr = expr, sense=pyo.maximize)

##Restrições horas
modelo.r1 = pyo.Constraint(expr=sum(A[0,i]*modelo.x[i] for i in range(n))<=b[0])
modelo.r2 = pyo.Constraint(expr=sum(A[1,i]*modelo.x[i] for i in range(n))<=b[1])
modelo.r3 = pyo.Constraint(expr=sum(A[2,i]*modelo.x[i] for i in range(n))<=b[2])

modelo.pprint()

In [10]:
!apt-get install coinor-cbc


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 23 not upgraded.
Need to get 2,875 kB of archives.
After this operation, 8,782 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 coinor-libcoinutils3v5 amd64 2.11.4+repack1-1 [480 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 coinor-libosi1v5 amd64 0.108.6+repack1-1 [294 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 coinor-libclp1 amd64 1.17.5+repack1-1 [937 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 coinor-libcgl1 amd64 0.60.3+repack1-2 [430 kB]
Get:5 htt

In [12]:
solver = pyo.SolverFactory("cbc")
results = solver.solve(modelo, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Mar 24 2020 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpg2v6trrq.pyomo.lp -stat=1 -solve -solu /tmp/tmpg2v6trrq.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
 CoinLpIO::readLp(): Maximization problem reformulated as minimization
Coin0009I Switching back to maximization to get correct duals etc
Presolve 3 (-1) rows, 2 (-1) columns and 6 (-1) elements
Statistics for presolved model


Problem has 3 rows, 2 columns (2 with objective) and 6 elements
Column breakdown:
2 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
0 of type E 0.0, 0 of type E 1.0, 0 of type E -1.0, 
0 of type E other, 0 of type G 0.0, 0 of type G 1.0, 
0 of type G other, 0 of type L 0.0, 0 of type L 1.0, 
3 of type L other, 0 of type Range 0.0->1.0, 0 of type Range 

In [7]:
print("Valor ótimo lucro = ", modelo.obj())
print("Solução ótima = ", modelo.x.get_values())

Valor ótimo lucro =  193.3333333333335
Solução ótima =  {0: 26.6666666666667, 1: 46.6666666666667}
